In [1]:
import contract_utilities.expiration as exp
import opportunity_constructs.intraday_future_spreads as ifs
date_list = exp.get_bus_day_list(date_from=20160901, date_to=20161011)

In [ ]:

# date_from=20160701, date_to=20160908

for i in range(len(date_list)):
        print(date_list[i])
        ifs.generate_ifs_sheet_4date(date_to=date_list[i])
        
# last 0719        

In [3]:
import backtesting.backtest_ifs as bifs
backtest_output_list = []
for i in range(len(date_list)):
        print(date_list[i])
        backtest_output_list.append(bifs.backtest_ifs_4date(report_date=date_list[i]))

20160808
20160809
20160810
20160811
20160812
20160815
20160816
20160817
20160818
20160819
20160822
20160823
20160824
20160825
20160826
20160829
20160830
20160831
20160901
20160902
20160906
20160907
20160908
20160909
20160912
20160913
20160914
20160915
20160916
20160919
20160920
20160921
20160922
20160923
20160926
20160927
20160928
20160929
20160930
20161003
20161004
20161005
20161006
20161007
20161010
20161011


In [4]:
import pandas as pd
backtest_results = pd.concat(backtest_output_list)
backtest_results['report_date']

pnl1_list = []
pnl2_list = []
pnl5_list = []
pnl6_list = []

pnl1_wc_list = []
pnl2_wc_list = []
pnl5_wc_list = []
pnl6_wc_list = []

for i in range(len(date_list)):
    daily_results = backtest_results[backtest_results['report_date'] == date_list[i]]
    pnl1_list.append(daily_results['pnl1'].mean())
    pnl2_list.append(daily_results['pnl2'].mean())
    pnl5_list.append(daily_results['pnl5'].mean())
    pnl6_list.append(daily_results['pnl6'].mean())
    pnl1_wc_list.append(daily_results['pnl1_wc'].mean())
    pnl2_wc_list.append(daily_results['pnl2_wc'].mean())
    pnl5_wc_list.append(daily_results['pnl5_wc'].mean())
    pnl6_wc_list.append(daily_results['pnl6_wc'].mean())

portfolio_pnl = pd.DataFrame.from_items([('report_date', date_list), 
                         ('pnl1', pnl1_list),
                         ('pnl2', pnl2_list),
                         ('pnl5', pnl5_list),
                         ('pnl6', pnl6_list),
                         ('pnl1_wc', pnl1_wc_list),
                         ('pnl2_wc', pnl2_wc_list),
                         ('pnl5_wc', pnl5_wc_list),
                         ('pnl6_wc', pnl6_wc_list)])    

In [30]:
import math as m
unique_spread_description_list = backtest_results['spread_description'].unique()

i= 5
selected_results = backtest_results[backtest_results['spread_description']==unique_spread_description_list[i]]
print(unique_spread_description_list[i])
print(selected_results['pnl6_wc'].mean())
print(m.sqrt(252)*selected_results['pnl6_wc'].mean()/selected_results['pnl6_wc'].std())

W_KW
682.5698255017234
5.245508340271369


In [5]:
import matplotlib.pyplot as plt
plt.plot(range(len(selected_results.index)),selected_results['pnl1'].cumsum(),
         range(len(selected_results.index)),selected_results['pnl1_wc'].cumsum())
plt.grid()         
plt.legend(['pnl6','pnl6_wc'],frameon=False)         
plt.show()

NameError: name 'selected_results' is not defined

In [25]:
import signals.intraday_futures_signals as ifs
import datetime as dt
import shared.calendar_utilities as cu

ticker_list = ['SX2016','BOZ2016','SMZ2016']
ticker_list = ['CLU2016','BU2016']
ticker_list = ['CU2016','WU2016']
ticker_list = ['WU2016','KWU2016']
ticker_list = ['RBQ2016','CLQ2016']
ticker_list = ['HOQ2016','CLQ2016']
report_date = 20160711
forward_date = [x for x in date_list if x>report_date][1]

signals_output = ifs.get_intraday_spread_signals(ticker_list=ticker_list,date_to=forward_date,num_days_back_4intraday=3)
intraday_data = signals_output['intraday_data']



start_hour = dt.time(9, 0, 0, 0)
end_hour = dt.time(12, 55, 0, 0)

selection_indx = [x for x in range(len(intraday_data.index)) if
                          (intraday_data.index[x].to_datetime().time() < end_hour)
                          and(intraday_data.index[x].to_datetime().time() >= start_hour)]

intraday_data = intraday_data.iloc[selection_indx]

intraday_data_yesterday = intraday_data[intraday_data['settle_date'] == cu.convert_doubledate_2datetime(report_date).date()]


intraday_mean1 = intraday_data_yesterday['spread'].mean()
intraday_std1 = intraday_data_yesterday['spread'].std()

import matplotlib.pyplot as plt
plt.plot(range(len(intraday_data.index)),intraday_data['spread'])
plt.axhline(intraday_mean1+intraday_std1,color='r')
plt.axhline(intraday_mean1-intraday_std1,color='r')
#plt.plot(range(len(intraday_data.index)),intraday_data['c1']['mid_p'])
#plt.plot(range(len(intraday_data.index)),intraday_data['c2']['mid_p'])

plt.grid()         
         
plt.show()

#intraday_data[intraday_data['spread']<-3]


In [6]:
import matplotlib.pyplot as plt
plt.plot(range(len(portfolio_pnl.index)),portfolio_pnl['pnl1'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl2'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl5'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl6'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl1_wc'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl2_wc'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl5_wc'].cumsum(),
         range(len(portfolio_pnl.index)),portfolio_pnl['pnl6_wc'].cumsum())
plt.grid()         
plt.legend(['pnl1','pnl2','pnl5','pnl6','pnl1_wc','pnl2_wc','pnl5_wc','pnl6_wc'],frameon=False)         
plt.show()


In [7]:
import math as m
print(m.sqrt(252)*portfolio_pnl['pnl1'].mean()/portfolio_pnl['pnl1'].std())
print(m.sqrt(252)*portfolio_pnl['pnl2'].mean()/portfolio_pnl['pnl2'].std())
print(m.sqrt(252)*portfolio_pnl['pnl5'].mean()/portfolio_pnl['pnl5'].std())
print(m.sqrt(252)*portfolio_pnl['pnl6'].mean()/portfolio_pnl['pnl6'].std())

print(m.sqrt(252)*portfolio_pnl['pnl1_wc'].mean()/portfolio_pnl['pnl1_wc'].std())
print(m.sqrt(252)*portfolio_pnl['pnl2_wc'].mean()/portfolio_pnl['pnl2_wc'].std())
print(m.sqrt(252)*portfolio_pnl['pnl5_wc'].mean()/portfolio_pnl['pnl5_wc'].std())
print(m.sqrt(252)*portfolio_pnl['pnl6_wc'].mean()/portfolio_pnl['pnl6_wc'].std())

8.499294412338827
7.992983746443655
5.92896345741934
8.848171866251363
6.390607341145249
5.941697867611258
4.353418570196607
6.651298463361353


In [7]:
portfolio_pnl['pnl1_wc']

0        0.000000
1      163.091158
2     2237.962109
3     1809.832457
4    -1699.986213
5     2833.966390
6     1252.750953
7    -1699.619044
8      331.585265
9      125.414446
10   -1050.487193
11     612.756834
12     419.074468
13     143.674685
14    1346.557254
15    -135.700653
16      90.315362
17     422.942793
18     547.405439
19     435.883016
20     137.512958
21    1887.172584
22    -237.240214
23    -113.924475
24     556.071863
25    1347.241473
26     182.327360
27     -17.817802
28     940.953785
29     494.338156
30     321.604861
31     794.815086
32    -327.797200
33     874.137788
34    -872.953162
35   -1315.379770
36     297.597431
37     714.188745
38    -192.129994
39     256.448172
40     866.154367
41   -1066.660872
42    -353.322789
43      59.942037
44     437.934506
45     186.725857
46     801.520805
47     714.361417
48    -452.596903
49     959.948404
50     329.935207
51   -1136.428664
52    1257.901508
53    -465.397922
54     565.617993
55     160